In [3]:
import pandas as pd
import json
from sklearn.cluster import KMeans
australian_users_items = pd.read_csv("aus_user_item.csv", index_col=[0]).reset_index()
items_json = json.load(open("numeric_values_tags.json"))
item_df = pd.DataFrame(items_json)
item_df["item_id"] = item_df["item_id"].astype(int)
item_tag_list = item_df.tags.tolist()

In [4]:
kmeans = KMeans(n_clusters=8, random_state=42, max_iter=600, init="k-means++").fit(item_tag_list)
item_label = {list(item_df.item_id)[i]:kmeans.labels_[i] for i in range(len(kmeans.labels_))}
item_df["label"] = item_df.apply(lambda x: item_label[x.item_id], axis=1)


In [5]:
# from sklearn.manifold import TSNE
# from matplotlib import pyplot as plt
# tsne = TSNE(n_components=2, random_state=0)
# user_2_dim = tsne.fit_transform(item_tag_list)

In [6]:
# import matplotlib.cm as cm
# import numpy as np
# fig, ax = plt.subplots()
# ax.scatter(user_2_dim[:, 0], user_2_dim[:, 1], c=kmeans.labels_, cmap='viridis')
# plt.show()

In [7]:
def set_label_id(item_id):
    try:
        return item_label[item_id]
    except:
        return -1 # 表示上一个表里面没有，需要调查
australian_users_items['item_label'] = australian_users_items.apply(lambda x: set_label_id(x["item_id"]), axis=1)
# australian_users_items = australian_users_items.drop("index", axis=1)
australian_users_items = australian_users_items[australian_users_items["item_label"] != -1]

In [8]:
user_label_count = australian_users_items.groupby("steam_id")["item_label"].nunique()
user_max_game_hours = australian_users_items.groupby("steam_id")["playtime_forever"].max()
user_total_game_hours = australian_users_items.groupby("steam_id")["playtime_forever"].sum()
user_features = pd.merge(pd.merge(user_total_game_hours,user_label_count, on="steam_id"), user_max_game_hours, on="steam_id")

In [9]:
user_features.rename({"playtime_forever_x":"user_total_game_hours", "playtime_forever_y":"user_max_game_hours"},axis=1, inplace=True)

In [10]:
user_feature = pd.concat([user_features, pd.DataFrame(columns=['user_depth','user_width'])], sort=False)
user_feature["user_width"] = user_feature.apply(lambda x: x['item_label'] / 8, axis=1)
user_feature['user_depth'] = user_feature.apply(lambda x: x['user_max_game_hours'] / x['user_total_game_hours'] if x['user_total_game_hours'] != 0 else 0, axis=1)
user_feature = pd.DataFrame(user_feature, columns=['user_depth', 'user_width'])
user_feature = user_feature.reset_index().rename({"index":"user_id"},axis=1)

In [11]:
user_feature.to_csv("features_dataset/user_features_with_depth.csv")

In [12]:
user_feature

,user_id,user_depth,user_width
0,76561197960269200,0.228855,0.875
1,76561197960270504,0.163600,0.875
2,76561197960270742,0.944925,0.500
3,76561197960271609,0.486380,0.250
4,76561197960271888,0.668127,0.625
...,...,...,...
68707,76561198327778018,1.000000,0.125
68708,76561198328421116,0.419540,0.125
68709,76561198328629223,0.000000,0.125
68710,76561198328946162,0.998454,0.250
